In [1]:
import pandas as pd
import os
from utils import run_reduced_scenarios, plot_uncertainty, nb_days

### What this script does ?

- Get hydrogeneration years
- Apply rules such as seasonal average to represent reservoirs management
- Find multiple representative years to be used in the optimization model
- Compute with capacity to calculate pAvailability in EPM format
- Export in .csv format

In [2]:
# All the inputs and output will be stored in the following folder
main_folder = 'data/guinea/repr_yrs'

In [3]:
rename_hpp = {
    'amaria': 'Amaria',
    'baneah': 'Baneah', 
    'bonkon_diaria': 'Bonkon-Diaria', 
    'boureya': 'Boureya', 
    'diallol': 'Diallol', 
    'diareguela': 'Diareguela', 
    'digan': 'Digan',
    'donkea': 'Donkea', 
    'farankonedou': 'Farankonedou', 
    'fello_sounga': 'Fello Sounga',
    'fetore': 'Fetore', 
    'fomi': 'Fomi', 
    'garafiri': 'Garafiri',
    'grand_chutes': 'Grand Chutes', 
    'grand_kinkon': 'Grand Kinkon',
    'guozoguezia': 'Guozoguezia',
    'hakkunde': 'Hakkunde-Mitti',
    'kaleta': 'Kaleta',
    'kassa': 'Kassa',
    'kogebedou': 'Kogebedou',
    'korafindi': 'Korafindi',
    'kouloutamba': 'Koukoutamba',
    'kouravel': 'Kouravel',
    'morisanako': 'Morisanako',
    'niagara': 'Niagara',
    'nzebela': 'Nzebela',
    'poudalde': 'Poudalde',
    'souapiti': 'Souapiti',
    'tiopo_105': 'Tiopo'
    
}

In [4]:
def calculate_seasonal_average(df):
    """Calculate the seasonal average of the data.
    
    The seasonal average is calculated for the period from November to June.
    
    Parameters
    ----------
    df : pd.DataFrame
        The data to calculate the seasonal average. The data should be in the format of a DataFrame with the columns
        'year' and 'month' and the values to calculate the seasonal average.
    
    Returns
    -------
    pd.DataFrame
    """

    temp = df.stack()
    temp.index.names = ['month', 'year']
    temp = temp.reorder_levels(['year', 'month'])
    temp = temp.sort_index()
    temp = temp.reset_index(name='value')
    temp['seasonal year'] = temp['year'].astype(int)
    temp.loc[temp['month'] >= 11, 'seasonal year'] += 1
    temp = temp.astype({'year': int, 'month': int, 'value': float, 'seasonal year': int})
    temp_filtered = temp[temp['month'].isin([11, 12, 1, 2, 3, 4, 5, 6])]
    seasonal_averages = temp_filtered.groupby('seasonal year')['value'].mean()
    seasonal_averages.name = 'value avg'
    temp_filtered = pd.merge(temp_filtered, seasonal_averages, left_on='seasonal year', right_index=True)
    temp_filtered.loc[:, 'value'] = temp_filtered.loc[:, 'value avg']
    temp_filtered = temp_filtered.loc[:, ['year', 'month', 'value']]
    temp_filtered.rename(columns={'value': 'value avg'}, inplace=True)

    temp = pd.merge(temp, temp_filtered, on=['year', 'month'], how='left')
    # Replace the value by the average value if value avg is not missing
    temp.loc[~temp['value avg'].isnull(), 'value'] = temp.loc[~temp['value avg'].isnull(), 'value avg']
    
    temp = temp.loc[:, ['year', 'month', 'value']]
    temp = temp.pivot(index='month', columns='year', values='value') 
    temp.columns = temp.columns.astype(str)

    return temp

In [15]:
# Read all csv file in the folder
n_clusters, method = 2, "real"
rename_scenario = None
impact_cc = None
extract_extreme = True
format_preparation = 'stochastic'
folder = 'data_hydro_guinea'
out_folder = 'output_{}'.format(folder)
suffix_file = '{}'.format(n_clusters)
if impact_cc is not None:
    suffix_file = '{}_cc'.format(n_clusters)
if extract_extreme:
    suffix_file = '{}_extreme'.format(suffix_file)

folder = os.path.join(main_folder, folder)


existing_plant = ['souapiti', 'kaleta', 'garafiri']

if impact_cc is not None:
    out_folder = out_folder + '_cc'
out_folder = os.path.join(main_folder, out_folder)
if not os.path.exists(out_folder):
    os.makedirs(out_folder)
files = [f for f in os.listdir(folder) if f.endswith('.csv')]
data, results = {}, {}
tot, tot_existing = [], []

if impact_cc is not None:
    impact_cc = pd.read_csv(os.path.join(main_folder, 'impact_cc_hydro_guinea.csv'), index_col=[0]).squeeze()

# Results is not used here as reprensentative years is determine based on total hydropower production
for f in files:
    print(f)
    name = f.split('.')[0]
    df = pd.read_csv(os.path.join(folder, f), index_col=[0])
    
    # only select years after 1980
    df.columns = pd.to_numeric(df.columns)
    df = df.loc[:, [i for i in df.columns if i >= 1980]]
    df.columns = df.columns.astype(str)

    # keep the average production for the dry season
    df = calculate_seasonal_average(df)
    
    if impact_cc is not None:
        df_cc = pd.DataFrame()
        for k, i in impact_cc.items():
            temp = df.copy() * (1 + i)
            temp.columns = ['{}-{}'.format(i, k)  for i in temp.columns]
            df_cc = pd.concat([df_cc, temp], axis=1)
            
        df = df_cc.copy()
    
    # transform from mw to gwh
    # df = (df.T * nb_days).T * 24 / 1e3
    data.update({name: df.copy()})
    # results.update({f.split('.')[0]: run_reduced_scenarios(df, n_clusters=n_clusters, method=method)})
    tot = tot + [df]
    if name in existing_plant:
        print(name)
        tot_existing = tot_existing + [df]
        
# Sum all the hydropower production
tot = sum(tot)
tot_existing = sum(tot_existing)

# indicator
# average monthly capacity (MW)
prod = (tot.T * nb_days).T * 24 / 1e3
indicator = {'yearly_prod_avg': prod.sum().mean(), 'monthly_prod_avg': prod.mean(axis=1)}

# Find the representative years
if extract_extreme:
    worst_case = tot.sum().sort_values().iloc[:int(len(tot.columns) / 20)]
    min_prod = {k: i.loc[:, list(worst_case.index)].mean(axis=1) for k, i in data.items()}
    data = {k: i.drop(columns=list(worst_case.index)) for k, i in data.items()}
    proba_min = worst_case.shape[0] / len(tot.columns)
    
    best_case = tot.sum().sort_values().iloc[-int(len(tot.columns) / 20):]
    max_prod = {k: i.loc[:, list(best_case.index)].mean(axis=1) for k, i in data.items()}
    data = {k: i.drop(columns=list(best_case.index)) for k, i in data.items()}
    proba_max = best_case.shape[0] / len(tot.columns)
    
years_repr = run_reduced_scenarios(tot, n_clusters=n_clusters, method=method)
scenarios = [i.split(' - ')[0] for i in years_repr.columns]

# Select the data for the representative years
data = {k: i.loc[:, scenarios].stack() for k, i in data.items()}
if format_preparation == 'monte_carlo':
    mc_folder = os.path.join(out_folder, 'output_monte_carlo')
    if not os.path.exists(mc_folder):
        os.makedirs(mc_folder)
    

elif format_preparation == 'stochastic':
    data = pd.concat(data, axis=1, names=['hpp']).T
    data.columns.names = ['Month', 'Scenarios']
    data = data.reorder_levels(['Scenarios', 'Month'], axis=1)
    data = data.sort_index(axis=1)

    # Assing the scenarios to low, medium and high
    if n_clusters == 1:
        rename_scenario = {scenarios[0]: 'baseline'}
        data.rename(columns=rename_scenario, level='Scenarios', inplace=True)
    elif n_clusters == 3:
        temp = tot.loc[:, scenarios].sum()
        rename_scenario = {temp.idxmin(): 'low', temp.idxmax(): 'high', [i for i in temp.index if i not in [temp.idxmin(), temp.idxmax()]][0]: 'medium'}
        data = data.rename(columns=rename_scenario, level='Scenarios')

    if extract_extreme:
        min_prod = pd.concat(min_prod, axis=1, names=['hpp']).T
        min_prod = pd.concat([min_prod], axis=1, keys=['min'])
        min_prod.columns.names = ['Scenarios', 'Month']
        min_prod = min_prod.sort_index(axis=1)
        
        data = pd.concat([data, min_prod], axis=1)
        
        max_prod = pd.concat(max_prod, axis=1, names=['hpp']).T
        max_prod = pd.concat([max_prod], axis=1, keys=['max'])
        max_prod.columns.names = ['Scenarios', 'Month']
        max_prod = max_prod.sort_index(axis=1)
        
        data = pd.concat([data, max_prod], axis=1)

    # Rename the hpp
    if rename_hpp is not None:
        data.rename(index=rename_hpp, level='hpp', inplace=True)
        
    capacity_file = 'capacity_hydro_guinea.csv'
    # Normalize by capacity to get the availability
    if capacity_file in os.listdir(main_folder):
        capacity_file = os.path.join(main_folder, capacity_file)

        print('Capacity file exist to calculate the availability')
        capacity = pd.read_csv(capacity_file, index_col=[0]).squeeze()
        capacity = capacity.loc[data.index.get_level_values('hpp').unique()]
        availability = (data.T / capacity).T
    else:
        raise ValueError('Capacity file is missing')

    # Export in EPM format

    availability.index.names = [None]
    
    availability_other = 'pAvailability_others.csv'
    if availability_other in os.listdir(main_folder):
        availability_other = os.path.join(main_folder, availability_other)
        
        print('Availability file exists for other sources')
        
        availability_other = pd.read_csv(availability_other, index_col=[0], header=[0])
        availability_other.columns.names = ['Month']
        availability_other.columns = availability_other.columns.astype(int)
        
        availability_other = pd.concat([availability_other] * len(availability.columns.get_level_values('Scenarios').unique()), keys=availability.columns.get_level_values('Scenarios').unique(), axis=1)
        availability = pd.concat((availability, availability_other), axis=0)
        
    # data.columns.names = [None, None]
    availability.round(3).to_csv(os.path.join(out_folder, 'pAvailability_hydro_{}.csv'.format(suffix_file)))

    # For sensitivity analysis
    if n_clusters == 3:
        for i in ['low', 'medium', 'high']:
            availability.loc(axis=1)[i].round(3).to_csv(os.path.join(out_folder, 'pAvailability_hydro_{}.csv'.format(i)))

    # Probabilities is in the name of the scenarios
    pProbaScenarios = pd.Series([i.split(' - ')[1] for i in years_repr.columns], index=pd.Index([i.split(' - ')[0] for i in years_repr.columns], name='Scenarios'), name='Value')
    if rename_scenario is not None:
        pProbaScenarios = pProbaScenarios.rename(index=rename_scenario)
    if extract_extreme:
        pProbaScenarios = pProbaScenarios.astype(float) * (1 - proba_min - proba_max)
        pProbaScenarios.loc['min'] = proba_min
        pProbaScenarios.loc['max'] = proba_max
        
    pProbaScenarios.to_csv(os.path.join(out_folder, 'pProbaScenarios_hydro_{}.csv'.format(suffix_file)))



fello_sounga.csv
farankonedou.csv
baneah.csv
poudalde.csv
nzebela.csv
tiopo_105.csv
niagara.csv
digan.csv
bonkon_diaria.csv
grand_chutes.csv
diallol.csv
amaria.csv
fetore.csv
kaleta.csv
kaleta
kouloutamba.csv
donkea.csv
fomi.csv
garafiri.csv
garafiri
guozoguezia.csv
hakkunde.csv
korafindi.csv
souapiti.csv
souapiti
kogebedou.csv
diareguela.csv
kouravel.csv
kassa.csv
morisanako.csv
grand_kinkon.csv
boureya.csv
Capacity file exist to calculate the availability
Availability file exists for other sources


## Checking the results

In [16]:
if 'baseline' in data.columns.get_level_values('Scenarios'):
    capacity_baseline = data.xs('baseline', level='Scenarios', axis=1)
    prod = (capacity_baseline * nb_days).T * 24 / 1e3
    display(prod.sum(axis=1), prod.sum().sum())

    prod_current = prod.loc[:, ['Kaleta', 'Souapiti']]
    display(prod_current.sum(axis=1))

In [19]:
prod = data.mul(nb_days, level='Month', axis=1) * 24 / 1e3
capacity = data.sum()
existing_plant = [rename_hpp[i] for i in existing_plant if i in rename_hpp]
capacity_existing = data.loc[existing_plant, :].sum()
prod = prod.sum()
prod, capacity, capacity_existing = prod.unstack('Scenarios'), capacity.unstack('Scenarios'), capacity_existing.unstack('Scenarios')

NameError: name 'xx' is not defined

Scenarios             1997                                                  \
Month                   1           2           3           4           5    
Fello Sounga     13.671000   12.348000   13.671000   13.230000   13.671000   
Farankonedou      9.672000    8.736000    9.672000    9.360000    9.672000   
Baneah            2.046000    1.848000    2.046000    1.980000    2.046000   
Poudalde         20.274000   18.312000   20.274000   19.620000   20.274000   
Nzebela          13.226502   11.946518   13.226502   12.799841   13.226502   
Tiopo            47.988000   43.344000   47.988000   46.440000   47.988000   
Niagara          15.438000   13.944000   15.438000   14.940000   15.438000   
Digan            11.346000   10.248000   11.346000   10.980000   11.346000   
Bonkon-Diaria    26.226000   23.688000   26.226000   25.380000   26.226000   
Grand Chutes     13.764000   12.432000   13.764000   13.320000   13.764000   
Diallol          22.878000   20.664000   22.878000   22.140000   22.878000   
Amaria           87.580094   79.104601   87.580094   84.754930   87.580094   
Fetore           10.323000    9.324000   10.323000    9.990000   10.323000   
Kaleta           87.978000   79.464000   87.978000   85.140000   87.978000   
Koukoutamba      61.194000   55.272000   61.194000   59.220000   61.194000   
Donkea            8.091000    7.308000    8.091000    7.830000    8.091000   
Fomi             37.386000   33.768000   37.386000   36.180000   37.386000   
Garafiri         22.413000   20.244000   22.413000   21.690000   22.413000   
Guozoguezia      33.759000   30.492000   33.759000   32.670000   33.759000   
Hakkunde-Mitti   10.974000    9.912000   10.974000   10.620000   10.974000   
Korafindi        40.548000   36.624000   40.548000   39.240000   40.548000   
Souapiti        166.284000  150.192000  166.284000  160.920000  166.284000   
Kogebedou        38.037000   34.356000   38.037000   36.810000   38.037000   
Diareguela       27.528000   24.864000   27.528000   26.640000   27.528000   
Kouravel         15.066000   13.608000   15.066000   14.580000   15.066000   
Kassa            35.154000   31.752000   35.154000   34.020000   35.154000   
Morisanako       31.434000   28.392000   31.434000   30.420000   31.434000   
Grand Kinkon     16.833000   15.204000   16.833000   16.290000   16.833000   
Boureya          41.385000   37.380000   41.385000   40.050000   41.385000   

Scenarios                                                                   \
Month                   6           7           8           9           10   
Fello Sounga     13.230000   17.112000   15.624000   10.080000   26.784000   
Farankonedou      9.360000   12.648000    9.672000   26.640000   23.064000   
Baneah            1.980000    2.232000    2.232000    2.880000    2.232000   
Poudalde         19.620000   31.248000   25.296000   59.040000   51.336000   
Nzebela          12.799841   16.972911   14.691527    9.090628   20.410316   
Tiopo            46.440000   34.224000   20.832000   15.840000    2.976000   
Niagara          14.940000   17.112000   19.344000   11.520000   38.688000   
Digan            10.980000   14.136000   16.368000   25.920000   37.200000   
Bonkon-Diaria    25.380000   48.360000   43.896000  121.680000   80.352000   
Grand Chutes     13.320000   18.600000   18.600000   18.000000   18.600000   
Diallol          22.140000   31.248000   26.040000   44.640000   49.104000   
Amaria           84.754930   90.850089   94.134523  174.420000  192.811596   
Fetore            9.990000   27.528000   26.040000   65.520000   37.200000   
Kaleta           85.140000  101.184000   80.352000  105.120000  151.776000   
Koukoutamba      59.220000   62.496000   72.168000   42.480000  124.992000   
Donkea            7.830000   10.416000   10.416000   10.080000   10.416000   
Fomi             36.180000   34.224000   25.296000   13.680000   39.432000   
Garafiri         21.690000   25.296000   20.832000   11.520000   39.432000   
Guozoguezia      32.6

In [8]:
filename = os.path.join(out_folder, 'production_hydro_tot_{}.png'.format(suffix_file))
plot_uncertainty(tot, df2=capacity, title="Hydropower capacity (MW)", ylabel="MW", filename=filename, ymax=4500)

filename = os.path.join(out_folder, 'production_hydro_existing_{}.png'.format(suffix_file))
plot_uncertainty(tot_existing, df2=capacity_existing, title="Hydropower capacity existing (MW)", ylabel="MW", filename=filename, ymax=1100)


In [ ]:
capacity_existing

In [9]:
# make_simple_plot(prod, 'Hydropower production', 'GWh')
# make_simple_plot(capacity, 'Hydropower capacity', 'MW')